**Analisis Exploratorio de Datos**

In [0]:
from pyspark.sql import SparkSession

In [0]:
# Contenedor bronze
str_container = "bronze"
str_storage_name = "stinetraficodeveastus2"
str_storage_path = f"abfss://{str_container}@{str_storage_name}.dfs.core.windows.net/"

str_file_lesionados = "07BaseDatosFallecidosLecionadosPNC2023.csv"
str_file_hechos = "08BaseDatosHechosTransitoPNC2023.csv"
str_file_vehiculos = "09BaseDatosVehiculosInvolucradosPNC2023.csv"

str_ruta_lesionados = str_storage_path + str_file_lesionados
str_ruta_hechos = str_storage_path + str_file_hechos
str_ruta_vehiculos = str_storage_path + str_file_vehiculos

In [0]:
print("str_storage_path = ", str_storage_path)
print("str_ruta_lesionados = ", str_ruta_lesionados)
print("str_ruta_hechos = ", str_ruta_hechos)
print("str_ruta_vehiculos = ", str_ruta_vehiculos)

In [0]:
# Listar todos los archivos en el contenedor bronze
archivos = dbutils.fs.ls(str_storage_path)

print("Archivos en el contenedor bronze:")
for archivo in archivos:
    print(f"{archivo.name} - {archivo.size} bytes")

In [0]:
df_lesionados_raw = spark.read.option('header', True)\
                        .option('inferSchema', True)\
                        .csv(str_ruta_lesionados)
df_lesionados_raw.columns

In [0]:
df_lesionados_raw = spark.read.option('header', True)\
                        .option('inferSchema', True)\
                        .csv(str_ruta_hechos)
df_lesionados_raw.columns

In [0]:
df_vehiculos_raw = spark.read.option('header', True)\
                        .option('inferSchema', True)\
                        .csv(str_ruta_vehiculos)
df_vehiculos_raw.columns

In [0]:
df_vehiculos_raw.select('estado_con').distinct().orderBy('estado_con').display()

In [0]:
df_aux_fall_les.select('hora_ocu').distinct().orderBy('hora_ocu').display()
df_aux_fall_les.select('zona_ocu').distinct().orderBy('zona_ocu').display()
df_aux_fall_les.select('edad_per').distinct().orderBy('edad_per').display()

In [0]:
df_ine_hora = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_hora")
df_ine_hora.display()

In [0]:
df_ine_g_hora5 = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_g_hora_5")
df_ine_g_hora5.display()

In [0]:
df_ine_tipo_eve = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_tipo_eve")
df_ine_tipo_eve.display()

In [0]:
%sql
show tables in cat_ine_trafico_dev.ine_victimas_depto;

In [0]:
df_ine_g_hora5.printSchema()

In [0]:
%sql
DESCRIBE TABLE EXTENDED cat_ine_trafico_dev.gold_ine_trafico_dev.ine_victimas_depto;

In [0]:
%sql
CREATE TABLE cat_ine_trafico_dev.bronze_ine_trafico_dev.ine_aux_fallecitos_lesionados 
( num_corre INT, anio_ocu INT, dia_ocu INT, hora_ocu STRING, id_g_hora BIGINT, id_g_hora_5 BIGINT, id_mes_ocurrio BIGINT, id_dia_sem_ocu BIGINT, id_depto_ocu BIGINT, id_mupio_ocu BIGINT, zona_ocu STRING, id_sexo_per BIGINT, edad_per STRING, id_g_edad_80ymas BIGINT, id_g_edad_60ymas BIGINT, id_edad_quinquenales BIGINT, id_mayor_menor BIGINT, id_tipo_veh BIGINT, id_marca_veh BIGINT, id_color_veh BIGINT, id_modelo_veh BIGINT, id_g_modelo_veh BIGINT, id_tipo_eve BIGINT, id_fall_les BIGINT, id_int_o_noint BIGINT) USING delta TBLPROPERTIES ( 'delta.enableDeletionVectors' = 'true', 'delta.feature.appendOnly' = 'supported', 'delta.feature.deletionVectors' = 'supported', 'delta.feature.invariants' = 'supported', 'delta.minReaderVersion' = '3', 'delta.minWriterVersion' = '7', 'delta.parquet.compression.codec' = 'zstd') 

In [0]:
%sql
SHOW CREATE TABLE cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados;

In [0]:
%sql
SHOW CREATE TABLE cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_depto_ocu;

In [0]:
%sql
SHOW CREATE TABLE cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_mes_ocu;

In [0]:
%sql
SHOW CREATE TABLE cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_fall_les;

In [0]:
%sql
SHOW CREATE TABLE cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_sexo_per;

In [0]:
# Leer la tabla Delta
df = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados")

# Obtener los valores distintos de zona_ocu
distinct_zona_ocu = df.select("zona_ocu").distinct().orderBy("zona_ocu")

# Mostrar los resultados
distinct_zona_ocu.display()

In [0]:
%sql
SELECT  DISTINCT  id_mes_ocurrio
FROM    cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados;

In [0]:
%sql
SELECT  TH_HECHOS.fl_fecha FECHA,
        DIM_MES.mes_mezcla MES,
        DIM_FALL_LES.fall_les FALLECIDOS,
        SUM(1) INCIDENTES
FROM    cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados   TH_HECHOS,
        cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_mes_ocu DIM_MES,
        cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_fall_les DIM_FALL_LES
WHERE   TH_HECHOS.id_mes_ocurrio = DIM_MES.id_mes_ocurrio
AND     TH_HECHOS.id_fall_les = DIM_FALL_LES.id_fall_les
GROUP BY fl_fecha, DIM_MES.mes_mezcla, DIM_FALL_LES.fall_les
ORDER BY FECHA;



In [0]:
%sql
SELECT  *
FROM    cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_fall_les 
LIMIT   50